# 라이브러리 설치

In [ ]:
!pip install -r requirements.txt

# 통신을 위한 클래스
* Jetson들과 통신하기 위한 ```JetsonSSHClient```
* Jetson들을 한번에 컨트롤하고 커맨드를 보내기 위한 ```JetsonController``` 

In [5]:
import paramiko
import time
import threading
import argparse
from tqdm import tqdm
min_port = 20101
max_port = 20101


class JetsonSSHClient:
  def __init__(self, port):
    self.port = port
    self.cli = paramiko.SSHClient()
    self.cli.set_missing_host_key_policy(paramiko.AutoAddPolicy)
    self.cli.connect("147.47.200.209", port=port, username="jetson", password="jetson")
    self.channel = self.cli.invoke_shell()
    
  def execute(self, command = "ls -al"):
    command = command +"\n"
    self.channel.send(command)
    # 결과 수신
    
    time.sleep(3) #wait for stdout
    output = self.channel.recv(65535).decode("utf-8")

    if self.port == 20101:
      print(output.strip(command))
    
  def close(self):
    self.cli.close()

class JetsonController:
  def __init__(self, min_port = 20101, max_port = 20101):
    self.clients = {}
    #self.threads = {}
    for port in tqdm(range(min_port, max_port + 1)):
      self.clients[port] = JetsonSSHClient(port)
      #self.threads[port] = None
    

  def global_execute(self, command = "ls -al"):
    for port, client in self.clients.items():
        try:
            print(f"=={port}==")
            client.execute(command)

        except Error as e:
            print(port, "port error")
            print(e)

# 시작전 서버 리셋

In [41]:
import requests
import json
client = 5
experiment = 4 
max_round = 5
init = requests.get(f"http://localhost:22222/initialize/{client}/{experiment}/{max_round}")
reset = requests.get("http://localhost:22222/reset")
print(init, init.text)

<Response [200]> Initialized server


# JetsonController                      

In [42]:
jetsonController = JetsonController(min_port=min_port, max_port=max_port)

  0%|          | 0/1 [00:02<?, ?it/s]


NoValidConnectionsError: [Errno None] Unable to connect to port 20101 on 147.47.200.209

# 모든 Jetson에 명령어를 보내기 위한 코드

In [7]:
while True:
    command = input()
    jetsonController.global_execute(str(command))

ls
Welcome to Ubuntu 18.04.5 LTS (GNU/Linux 4.9.201-tegra aarch64)

 * Documentation:  https://help.ubuntu.com
 * Management:     https://landscape.canonical.com
 * Support:        https://ubuntu.com/advantage
This system has been minimized by removing packages and content that are
not required on a system that users do not log into.

To restore this content, you can run the 'unminimize' command.

70 updates can be applied immediately.
35 of these updates are standard security updates.
To see these additional updates run: apt list --upgradable

Last login: Fri Sep 24 11:04:37 2021 from 147.47.7.75
jetson@jetson-desktop:~$ ls
ambient_fl  Downloads         jetson             Pictures   Videos
Desktop     examples.desktop  jetson_network.sh  Public
Documents   FL                Music              Templates
jetson@jetson-desktop:~$ 
docker attach fl

ls

exit
close

close
^CTraceback (most recent call last):
  File "jetson_client.py", line 420, in <module>
    client.task()
  File "jetson_

KeyboardInterrupt: Interrupted by user